In [1]:
import pandas as pd
from tqdm import tqdm

## 1st Step: Import DataFrames

In [2]:
df_content = pd.read_csv('data/banijay_op1data_content_processed.csv')

df_ratings = pd.read_csv('data/banijay_op1data_ratings_raw.csv', compression = 'zip')


/var/folders/7x/pkl50ghj76g8t2s0y2s5bcrw0000gn/T/ipykernel_69718/2904429814.py:3: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ratings = pd.read_csv('data/banijay_op1data_ratings_raw.csv', compression = 'zip')


## 2nd Step: Add date_time columns

In [3]:
df_content['content_id'] = df_content['id'].apply(lambda x: str(x)[14:28])

df_content['date_time_start_string'] = df_content['date'].apply(lambda x: str(x)[6:10]) + '-' +  df_content['date'].apply(lambda x: str(x)[3:6]) +  df_content['date'].apply(lambda x: str(x)[0:2]) + ' ' + df_content['start'].apply(lambda x: str(x)[0:6] +'00')

df_content['date_time_end_string'] = df_content['date'].apply(lambda x: str(x)[6:10]) + '-' +  df_content['date'].apply(lambda x: str(x)[3:6]) +  df_content['date'].apply(lambda x: str(x)[0:2]) + ' ' + df_content['end'].apply(lambda x: str(x)[0:6] +'00')


df_ratings['date_time_string'] = df_ratings['Datum'] + ' ' + df_ratings['Time']

In [4]:
df_content['date_time_start'] = df_content['date_time_start_string'].apply(lambda x: pd.to_datetime(x, format = '%Y-%m-%d %H:%M:%S'))

df_content['date_time_end'] = df_content['date_time_end_string'].apply(lambda x: pd.to_datetime(x, format = '%Y-%m-%d %H:%M:%S'))

In [5]:
df_ratings['date_time'] = df_ratings['date_time_string'].apply(lambda x: pd.to_datetime(x, format = '%Y-%m-%d %H:%M:%S'))

## 3rd Step: Check if the date_time is between the start and the end of the show

In [6]:
import numpy as np

def get_content_id(time):
   return df_content.loc[(df_content['date_time_start'] <= time) & (df_content['date_time_end'] >= time),'content_id']


df_ratings['content_id'] = [get_content_id(x) for x in tqdm(df_ratings['date_time'])]

df_ratings['content_id'] = df_ratings['content_id'].apply(lambda x: x.values)

100%|██████████| 1304190/1304190 [05:10<00:00, 4203.59it/s]


## 4th Step: Clean and merge final data

In [10]:

def drop_missing_values(x):
   if len(x) > 0:
      return x[0]
   else:
      return np.nan

df_ratings['content_id'] = df_ratings['content_id'].apply(lambda x: drop_missing_values(x))

df_ratings = df_ratings.drop(['Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0'],axis=1)

df_ratings.head(1000)

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,Datum,Time,Program,Station,Target Group,Broadcast Type,Ratings Type,Kdh%,Kdh000,Zadl%,date_time,year,date_time_str,content_id,date_time_string
0,0,0,0,0,2020-01-06,22:18:00,op1,npo1,tot6plus,live/prerecorded uitzendingen,uitzenddag,9.676969,1546.863437,30.881672,2020-01-06 22:18:00,2020-01,2020-01-06 22:18:00,NaN,2020-01-06 22:18:00
1,1,1,1,1,2020-01-06,22:18:00,op1,npo1,tot6plus,live/prerecorded uitzendingen,uitgesteld,1.484044,237.224411,24.692727,2020-01-06 22:18:00,2020-01,2020-01-06 22:18:00,NaN,2020-01-06 22:18:00
2,2,2,2,2,2020-01-06,22:18:00,op1,npo1,tot6plus,live/prerecorded uitzendingen,totaal,10.299407,1646.360210,29.962268,2020-01-06 22:18:00,2020-01,2020-01-06 22:18:00,NaN,2020-01-06 22:18:00
3,3,3,3,3,2020-01-06,22:18:00,op1,npo1,boodschapper_20_49,live/prerecorded uitzendingen,uitzenddag,3.217735,117.833443,14.746563,2020-01-06 22:18:00,2020-01,2020-01-06 22:18:00,NaN,2020-01-06 22:18:00
4,4,4,4,4,2020-01-06,22:18:00,op1,npo1,boodschapper_20_49,live/prerecorded uitzendingen,uitgesteld,1.146627,41.989484,17.021271,2020-01-06 22:18:00,2020-01,2020-01-06 22:18:00,NaN,2020-01-06 22:18:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,995,995,995,2020-01-06,22:51:00,op1,npo1,boodschapper_20_49,live/prerecorded uitzendingen,totaal,2.736553,100.212577,15.821852,2020-01-06 22:51:00,2020-01,2020-01-06 22:51:00,NaN,2020-01-06 22:51:00
996,996,996,996,996,2020-01-06,22:51:00,op1,npo1,boodschapper_25_54,live/prerecorded uitzendingen,uitzenddag,3.366896,134.877851,16.564046,2020-01-06 22:51:00,2020-01,2020-01-06 22:51:00,NaN,2020-01-06 22:51:00
997,997,997,997,997,2020-01-06,22:51:00,op1,npo1,boodschapper_25_54,live/prerecorded uitzendingen,uitgesteld,1.157118,46.354140,29.135196,2020-01-06 22:51:00,2020-01,2020-01-06 22:51:00,NaN,2020-01-06 22:51:00
998,998,998,998,998,2020-01-06,22:51:00,op1,npo1,boodschapper_25_54,live/prerecorded uitzendingen,totaal,3.489813,139.801896,16.178732,2020-01-06 22:51:00,2020-01,2020-01-06 22:51:00,NaN,2020-01-06 22:51:00


In [17]:
df_merged_2 = df_content.merge(df_ratings,on='content_id',how='left')

df_merged_2.to_csv('data/banijay_op1data_content_ratings.csv' , compression='zip')